In [1]:
from typing import List
from crewai import Agent, Crew, Process, Task, LLM
# from crewai.project import CrewBase, agent, crew, task

from crewai_tools import SerperDevTool, ScrapeWebsiteTool

import os

In [2]:
XAI_API_KEY = os.getenv("XAI_API_KEY")

llm = LLM(
    model="grok-beta",
    base_url="https://api.x.ai/v1",
    api_key=XAI_API_KEY,
)
# llm=LLM(model="ollama/crewai-llama3:latest", temperature=0.7)

In [5]:
class AgentSea:
    def __init__(self):
        self.llm = llm
    def ticker_finder_agent(self, company_name: str):
        return Agent(
            role="Senior Online Researcher",
            goal=
                """
                Conduct an amazing online search to find the stock ticker for {company_name}.
                IMPORTANT:
                - If you cannot find the result, return "Ticker not found".
                - Do not include any unnecessary words or phrases in your response and only provide the ticker symbol if found.
                """,
            backstory=
                """
                You are a Senior Online Researcher that specializes in finding a company's stock martket ticker symbol with only the company name.
                """,
            max_iter=5,
            max_retry_limit=3,
            tools=[SerperDevTool(), ScrapeWebsiteTool()],
            llm=self.llm,
            verbose=True,
            memory=False
        ),

In [6]:
class AgentTasks:
    def __init__(self, job: str):
        self.job = job
    
    def find_correct_company_ticker(self, agent: Agent, tasks: list[Task], company_name: str) -> Task:
        return Task(
            description=(
                "Conduct a thorough research about the company and find the correct stock market ticker symbol for {company_name}."
            ),
            agent=agent,
            expeceted_output_type=(
                "A string object containing only the ticker symbol for {company_name}."
            ),
            context=tasks,
        )

In [7]:
inputs = {
'company_name': 'tesla'
}

agent_sea = AgentSea()
tasks = AgentTasks("Test")
ticker_agent = agent_sea.ticker_finder_agent(inputs['company_name'])
task = tasks.find_correct_company_ticker(ticker_agent, tasks, inputs['company_name'])


2024-10-24 06:16:15,256 - 15000 - llm.py-llm:178 - ERROR: Failed to get supported params: argument of type 'NoneType' is not iterable



Provider List: https://docs.litellm.ai/docs/providers



AttributeError: 'tuple' object has no attribute 'get'

In [ ]:
crew = Crew(
			agents=[ticker_agent], # Automatically created by the @agent decorator
			tasks=[task], # Automatically created by the @task decorator
			process=Process.sequential,
			verbose=True,
		)

In [ ]:

result = crew.kickoff(inputs=inputs)